<a href="https://colab.research.google.com/github/QuaternionPaul/my-first-repo/blob/main/1_D_Ising_Model_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS267A Project

# Simulation of the 1-D Ising Model

The code for this simulation of the 1-D Ising Model/Ising Chain was adapted from [here](https://hockygroup.hosting.nyu.edu/exercise/ising-1d.html) [2].

The goal of this investigation was to demonstrate that the computationally expensive Markov Chain Monte Carlo (MCMC) step (for the Metropolis-Hastings algorithm) could be simply replaced by a probabilistic programming language (PPL) with similar results. Otherwise we use the same visualization backend and framework to display and compare the results.

In the first half of this notebook, we look at an implementation of the 1-D Ising chain from scratch without the use of a PPL. We show that even for the (relatively simple) Metropolis-Hastings MCMC algorithm, this is a rather complex affair. We also explore the expected physical behaviour of the 1-D Ising chain high/low temperature regimes.

In the second half, we build upon this framework by reimplementing the entire 1-D Ising Chain model with MCMC step in a PPL, namely PyMC3. We show that we can get similar results with much simpler and fewer lines of code in the PPL, demonstrating the potential of PPLs to speedup workflows when modeling probabilistic pehnomena. Our PPL implementation of the 1-D Ising chain also behaves as expected in the high/low temperature regimes.

In [ ]:
#setup the notebook
%pylab inline
import numpy as np


Populating the interactive namespace from numpy and matplotlib
time: 7.18 ms (started: 2023-06-16 08:11:46 +00:00)


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['step', 'trace']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


We generate a few simple 1-D Ising chains, specified to be of length 10 (aka 10 lattice sites).

As a convention, we choose **+1** to denote **spin-up** and **-1** to denote **spin-down** spins.

In [ ]:
def energy_ising_1d(configuration,J,h):
    num_spins = len(configuration)
    energy = 0.0
    for i in range(num_spins):
        spini = configuration[i]
        #set the value of spin i+1, make sure to test if i+1<num_spins, and otherwise account for periodic boundaries
        #you can do this with an if statement if you have to
        ip1 = (i+1)%num_spins
        spinip1 = configuration[ip1]

        energy = energy - J * (spini * spinip1) - h*spini

    return energy

#Check that the energy is correct
test_num_spins = 10
#this should be true for any J, h
test_J = 1
test_h = 2

test_configuration_1 = -1*np.ones(test_num_spins)
test_configuration_2 = +1*np.ones(test_num_spins)

test_configuration_3 = +1*np.ones(test_num_spins)
#this sets even entries to -1
test_configuration_3[::2] = -1

print("Test Config 1:", test_configuration_1)
print("Energy Config 1:", energy_ising_1d(test_configuration_1,test_J,test_h))
print("Expected Energy Config 1:",test_num_spins*(test_h-test_J))
print()
print("Test Config 2:", test_configuration_2)
print("Energy Config 2:", energy_ising_1d(test_configuration_2,test_J,test_h))
print("Expected Energy Config 2:",-test_num_spins*(test_h+test_J))
print()
print("Test Config 3:", test_configuration_3)
print("Energy Config 3:", energy_ising_1d(test_configuration_3,test_J,test_h))
print("Expected Energy Config 3:",test_num_spins*test_J)

Test Config 1: [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
Energy Config 1: 10.0
Expected Energy Config 1: 10

Test Config 2: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Energy Config 2: -30.0
Expected Energy Config 2: -30

Test Config 3: [-1.  1. -1.  1. -1.  1. -1.  1. -1.  1.]
Energy Config 3: 10.0
Expected Energy Config 3: 10
time: 1.99 ms (started: 2023-06-16 08:11:47 +00:00)


# Section 1
## Monte Carlo Simulation using Metropolis-Hastings algorithm (Non-PPL)

N.B: The below code was adapted from [here](https://hockygroup.hosting.nyu.edu/exercise/ising-1d.html) [2], solely for comparision purposes to show how the Metropolis-Hastings algorithm can be implemented manually without the use of a PPL.

We analyse the 1-D Ising chain using Monte Carlo simulation, but first without using a PPL. The Metropolis-Hastings MCMC algorithm is performed in selecting whether or not to flip individual spin-flips.

We also explore how the 1-D Ising chains behave under low and high temperature regimes, and see if the simulated behaviour agrees with theory.

In [ ]:


#set a seed for the random number generator here. For a given seed, all of your results should be identical
random_seed = 1
np.random.seed(random_seed)

def energy_difference(J, h, si, sleft, sright):
    #fill in the formula for the energy difference from flipping spin i,
    #which has value si= 1 or -1, with spin values sleft and sright on the left and right
    dE = 2*h*si + 2*J*si*(sleft+sright)
    return dE

def metropolis_mc_fast(n_steps, n_lattice_sites, beta, J, h, debug=False,save_freq=10):
    # we can start with a random configuration of size n_lattice_sites by generating a random list
    #    of zeros and twos, then subtracting 1, the following does that, do you see why? Play around
    #   with this function in an empty box if you don't
    configuration = 2*np.random.randint(2, size=n_lattice_sites) - 1
    average_spins = []

    if debug is True:
        print("Starting configuration:",configuration)

    current_energy = energy_ising_1d(configuration,J,h)
    for i in range(n_steps):

        #set this like you did above:
        spin_to_change = np.random.randint(n_lattice_sites)

        si = configuration[spin_to_change]
        #now figure out the values of the spin to the left and the spin to the right, remembering
        #  to take into account periodic boundary conditions for both
        # you can use if statements if you have to
        sright = configuration[(spin_to_change+1)%n_lattice_sites]
        sleft = configuration[(spin_to_change-1)%n_lattice_sites]

        dE = energy_difference(J,h,si,sleft,sright)

        r = np.random.random()
        # fill in the metropolis critereon, using dE
        if r<min(1,np.exp(-beta*(dE))) :
            #flip the spin
            configuration[spin_to_change]*=-1
            # update the energy
            current_energy += dE
        else:
            pass

        #this computes the average spin
        average_spin = configuration.mean()

        if i%save_freq == 0:
            average_spins.append(average_spin)

        if debug and i%10==0:
            print("%i: "%i,configuration,"Energy:",current_energy,"Spin:",average_spin)

    return average_spins

#do a test high temperature simulation
print("High temperature:")
average_spins = metropolis_mc_fast(n_steps=100, n_lattice_sites=10, beta=0.1, J=1, h=2, debug=True)
#do a test on a low temperature simulation
print("Low temperature:")
average_spins = metropolis_mc_fast(n_steps=100, n_lattice_sites=10, beta=1, J=1, h=2, debug=True)



High temperature:
Starting configuration: [ 1  1 -1 -1  1  1  1  1  1 -1]
0:  [-1  1 -1 -1  1  1  1  1  1 -1] Energy: -6.0 Spin: 0.2
10:  [-1  1  1 -1 -1  1  1  1  1  1] Energy: -10.0 Spin: 0.4
20:  [ 1 -1  1  1  1 -1  1 -1  1  1] Energy: -6.0 Spin: 0.4
30:  [ 1 -1 -1  1  1  1 -1 -1  1  1] Energy: -6.0 Spin: 0.2
40:  [-1 -1 -1 -1  1  1  1 -1 -1  1] Energy: 2.0 Spin: -0.2
50:  [-1  1 -1  1 -1  1 -1 -1 -1 -1] Energy: 10.0 Spin: -0.4
60:  [-1  1  1  1  1  1  1  1  1 -1] Energy: -18.0 Spin: 0.6
70:  [-1 -1 -1 -1 -1 -1  1 -1  1 -1] Energy: 10.0 Spin: -0.6
80:  [ 1  1  1  1 -1 -1  1  1 -1 -1] Energy: -6.0 Spin: 0.2
90:  [-1  1  1 -1  1 -1 -1  1  1 -1] Energy: 2.0 Spin: 0.0
Low temperature:
Starting configuration: [-1 -1  1 -1 -1 -1  1 -1  1  1]
0:  [ 1 -1  1 -1 -1 -1  1 -1  1  1] Energy: 2.0 Spin: 0.0
10:  [ 1 -1  1  1  1  1  1  1  1  1] Energy: -22.0 Spin: 0.8
20:  [1 1 1 1 1 1 1 1 1 1] Energy: -30.0 Spin: 1.0
30:  [1 1 1 1 1 1 1 1 1 1] Energy: -30.0 Spin: 1.0
40:  [1 1 1 1 1 1 1 1 1 1] Ene

**Remarks:**

The generated 1-D Ising chains' behaviour agree well with theory, under the low and high temperature regimes [1]. At low temperatures, the spins are all aligned (all spin-up, +1) and the spin-chain is highly ordered. At high temperatures, the stronger thermal excitations can randomly excite and flip individual spins, leading to a very disordered chain (random spin-up and spin-down, +1 or -1).

It is clear that the above non-probabilistic code is highly non-optimised with multiple nested for-loops, and probably would scale poorly in terms of computation as the length of the 1-D Ising chain (n_lattice_site) and number of simulated steps (n_steps) are increased. Additionally the implementation of even a relatively simple Metropolis-Hastings MCMC algorithm was not straight-forward. For more advanced/complicated MCMC methods like No U-Turn Sampling (NUTS) or Hamiltonian Monte Carlo (HMC), such methods may be very difficult and tme-consuming to implement from scratch.

 In the following segment, we build upon this framework to replicate this basic simulation of a 1-D Ising chain, albeit in a novel implementation using a PPL, namely PyMC3. We show that using a PPL results in much cleaner and simpler code, by moving the inference steps to be computed automatically in the backend and allowing the user to focus entirely on the modeling of the probabilistic system using PPL library functions.


#Section 2

#Simulation of 1-D Ising Chain using a PPL (PyMC3)

The following code adapts ideas from [here](https://tanyaschlusser.github.io/posts/mcmc-and-the-ising-model/), where a visualisation of the phase transition in the 2-D Ising model using the PyMC3 PPL is shown.

However we build upon this to make several novel technical contributions:

(1) extending the original 2-D framework to demonstrate how PyMC3 can be applied in the novel context of a 1-D Ising Chain.

(2) extending the total Hamiltonian to also consider the energy from interaction with a non-zero external magnetic field $h$

In this regard, the original code simplified the Hamiltonian by ignoring $\mathscr{H}^{1}_{Ext}$. We modify the function to be able to calculate the energy changes by including this new term in the total 1-D Hamiltonian:

\begin{equation}
    \mathscr{H}_{1D} = \mathscr{H}^{1}_{NN} + \mathscr{H}^{1}_{Ext} = \sum \limits^{N-1} \limits_{i=0} - J\sigma_{i}\cdot\sigma_{i+1} - h\cdot\sigma_{i}
\end{equation}

(3) refactor the implementation to use supported versions of the PyTensor and PyMC3 libraries. The original code used the Theano library, which has been discontinued in 2020 and had issues with being installed due to source errors.

We show that our implementation (in a PPL) is much shorter and simpler (in terms of lines of code) than the equivalent non-probabilistic code, with vectorized functions provided under the PyMC3 library removing the need for multiple-nesting of for loops as in the non-probabilistic implementation. The PyMC3 implementation also demonstrates expected features and behaviour for a 1-D Ising Chain, in good agreement with the non-probabilistic implementation above.

In [ ]:
from functools import partial

import pymc as pm
import numpy as np
import pytensor.tensor as pt

In [ ]:
def convert_spins (spins):

  # Bernoulli distributions outputs array of 1s and 0s
  # This fn converts original spins-array of 1s and 0s -> 1s and -1s
  # +1 represents spin-up, -1 represents spin-down

  proper = 2*spins - 1

  return proper


def get_H_NN(spins, J=1):

  # Calculate nearest-neighbour interaction Hamiltonian, H_NN

  spins_p = convert_spins(spins)

  H_NN = - J*(
  pt.roll(spins_p, 1, axis=0) * spins +
  pt.roll(spins_p, -1, axis=0) * spins
  )
  return H_NN

def get_H_Ext(spins, h=1):

  # Calculate external magnetic field interaction Hamiltonian, H_Ext

  H_ext = -h*spins

  return H_ext

def get_boltz_weight(spins, T=1):

  spins_p = convert_spins(spins)

  H_total = get_H_NN(spins) + get_H_Ext(spins)

  # WLOG, assume Boltzmann constant k_B = 1
  boltzmann_weight = pm.math.exp(-H_total/T)

  return boltzmann_weight


PyMC3 is a Python-based package that is focused on probabilistic machine learning and Bayesian modeling.

PyMC3 allows users access to a rich library of vectorized functions and statistical packages, that makes statistical inference tasks much more straight-forward and faster as compared to manual implementation. PyMC3 offers good support and access to powerful MCMC algorithms (Metropolis-Hastings, Sequential Monte Carlo, Hamiltonian Monte Carlo etc) and variational inference methods.

This opens up exciting possibilities for users to gain access to powerful new statistical methods such as No U-Turn Sampling (NUTS) with just a single function call from PyMC3, greatly speeding up programming workflows.

In the following, we demonstrate how the Metropolis-Hastings algorithm for Monte Carlo simulation of the 1-D Ising chain can just be built in a simple PyMC3.Model() class, with just a few lines of code.

The below snippet is an annotated version of our pymc.Model() which explains each relevant line of code in detail.

In [ ]:
basic_model = pm.Model()


with basic_model as model:
  T=100 # Temperature T will be defined as an argument in wrapper function.

  # Use the Bernoulli Distribution, to generate Binary random variables
  # Outputs an array of random 1s & 0s, array of shape/length 10
  y = pm.Bernoulli('y', 0.5, shape=10)

  # pm.Potential is summed and added to the overall log-likelihood.
  # This conditions the MCMC chain on the given argument function, which is the Boltzmann weighting
  boltz_weight = pm.Potential('b', -(get_H_NN(convert_spins(y)) + get_H_Ext(convert_spins(y)))/T )

  # `scaling` limits/control the fraction of positions in `x`
  # that flip at every step in the Markov Chain, to prevent over/under convergence
  # From the PyMC3 source:
  #     p_jump = 1. - .5 ** self.scaling
  #     ...
  #     switch_locs = (rand_array < p_jump)
  # The scaling hyperparameter was chosen to be 0.08 as we found it to give good results
  scaling = 0.08

  # PyMC3 allows us to conveniently choose the MCMC sampler method by calling the  supported function.
  # Other supported MCMC methods include Sequential MC, Hamiltonian MC, NUTS, can be just called directly
  step = pm.BinaryMetropolis([y], scaling=scaling)

  # Run the sample, store all outputs in trace
  trace = pm.sample(5000, step=step, return_inferencedata=False)

We embed this PyMC3 model within a wrapper function **ppl_approach()**, that allows us to conveniently invoke and perform inference on (n_steps) number of MCMC steps, on 1-D Ising chains of arbitrary length (n_lattice_sites).

---



In [ ]:
def ppl_approach(n_steps, n_lattice_sites, T, J=1, h=0, save_freq=10):

  basic_model = pm.Model()

  with basic_model as model:
    y = pm.Bernoulli('y', 0.5, shape=n_lattice_sites)

    boltz_weight = pm.Potential('b', -(get_H_NN(convert_spins(y)) + get_H_Ext(convert_spins(y)))/T )

    scaling = 0.008

    step = pm.BinaryMetropolis([y], scaling=scaling)

    trace = pm.sample(n_steps, step=step, return_inferencedata=False)

  output = 2*trace["y"][::save_freq*2]-1
  print(output)


We call the function to take n_steps = 5000 iterations of the MCMC algorithm, to quickly generate 5000 instances of 10-site 1-D Ising chains.

By default, PyMC3 samples two independent MCMC chains, in order to ensure convergence. One can change the number of chains by passing the "sample =" argument in the **sample()** function.

(The API note for the sample function is from: https://www.pymc.io/projects/docs/en/stable/api/generated/pymc.sample.html)

In [ ]:
print("Low Temperature: ")

ppl_approach( 5000, 10, T=1, J=1, h=1, save_freq = 500)

Low Temperature: 


[[1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]]


In [ ]:
print("High Temperature: ")

ppl_approach( 5000, 10, T=10, J=1, h=1, save_freq = 500)


High Temperature: 


[[-1 -1 -1  1  1  1 -1  1  1 -1]
 [ 1 -1 -1 -1 -1 -1 -1 -1  1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1  1 -1 -1 -1 -1 -1]
 [ 1  1 -1  1  1  1  1  1  1  1]
 [ 1  1  1  1 -1 -1 -1  1 -1 -1]
 [-1 -1  1  1 -1  1 -1 -1 -1 -1]
 [-1 -1  1  1  1  1 -1 -1 -1 -1]
 [-1 -1  1 -1 -1  1 -1 -1 -1  1]
 [-1  1 -1 -1 -1 -1 -1 -1  1  1]]


**Remarks:**

Our PyMC3 implementation of Monte-Carlo simulation of the 1-D Ising Chain agrees well with theory and the previous non-probabilistic implementation. Using a PPL results in much cleaner and straight-forward code than implementing the probabilistic model from scratch.

Our implementation is also more general, as we consider the extra added term of the Hamiltonian of the interaction of individual spins with an external magnetic field $h=1$, $\mathscr{H}^{1}_{Ext} = \sum \limits^{N-1} \limits_{i=0} - h\cdot\sigma_{i}$, which was absent in the non-PPL implementation in Section 1.

We also sees that the above implementation behaves as expected.

For an arbitrary **Low Temperature** (T=1), we see that all the spin-values = +1, denoting that all the spins are aligned.

For an arbitrary **High Temperature** (T=10), we see that the spin-values are largely random and highly-disordered, switching randomly between -1 and +1.

This behaviour is expected and agrees well with theory, and also the previous non-PPL implementation shown in Section 1. At low temperatures, the spins will largely be aligned with the external magnetic field $h$ with spin = +1, as random thermal excitations are too weak and have low probabiility of flipping the spin during the MCMC chain. At high temperatures the stronger thermal fluctuations/excitations lead to high probability of flipping of the spin-flips during the MCMC chain. Hence the spin-flips will be largely disordered and random at the end.

PyMC3 supports an extensive list of MCMC based samplers: https://www.pymc.io/projects/docs/en/stable/api/samplers.html

The user can just change the sampling algorithm in ppl_approach() by swapping the smapling function, without having to rewrite the entire MCMC function froms scratch.


# Section 3

## Further Work

Further work to explore the power of the PyMC3 PPL was also briefly explored, such as quantifying the runtime speedup of the PPL vs non-PPL implementation of the Ising model, using the **timeit()** Python module. However given limited manpower/time (as this was a solo project), our results were unconclusive (both showing comparable timings) and would need collection of more data and experiments.

Additionally, when testing the PyMC3 implementation, we realised that as the function works by running automatic probailistic inference on at least 2 independent MCMC chains in order to ensure convergence, it was thus doing at least twice the number of MCMC steps versus the non-PPL implementation. Hence our initial runtime tests probably did not evaluate the PPL approach fairly, and more consideration about how to design a fairer runtime comparison test between the non-PPL and PPL approaches is required.

However, we hypothesize that due to the highly optimised and vectorised nature of the PyMC3 library, the PPL approach would scale better with data size and be able to show significant runtime speedups. This could be tested by running a fair runtime comparison test between the non-PPL and PPL implementations on a very large 1-D Ising chain, for example one with n_lattice_sites = $10^{3}-10^{5}$, and observe any divergence in runtime scaling.

#References

[1] "One and two-dimensional Ising model". https://www.thphys.uni-heidelberg.de/~wolschin/statsem21_3s.pdf

[2] "Monte Carlo, 1-D Ising Model". https://hockygroup.hosting.nyu.edu/exercise/ising-1d.html

[3] "MCMC and the Ising Model" https://tanyaschlusser.github.io/posts/mcmc-and-the-ising-model/